In [1]:
from dotenv import load_dotenv
import os

# .env 불러오기
load_dotenv()

# 기본 모델 설정
# 여기서 설정할 수도 있지만 .env 내에 넣어줘도 됨
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Agent, Task

# Agent의 role, goal, backstory를 반드시 설정해줘야 함
international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    # agent가 무슨 생각을 하는지 알고 싶을 때 아래와 같이 설정
    verbose=True,
    # agent는 task를 생성할 수 있으며, 결정을 내리고 다른 agent들과 대화하거나 질문을 할 수 있음
    # 아래 설정은 다른 agent와 소통하지 않고 혼자서 역할을 해야 함을 설정하는 것
    allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [3]:
# Task 생성
normal_recipe = Task(
    # description: 어떤 작업을 시킬 것인지 설정
    description="Come up with a {dish} that serves {people} people.",
    # 어떤 agent에게 일을 시킬 것인지 설정
    agent=international_chef,
    # 예상 출력값 설정: 더 정확한 결과를 얻을 수 있음
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    # 파일 저장 설정
    output_file="normal_recipe.md",  # 마크다운 파일로 저장
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [5]:
crew = Crew(
    # CrewAI는 이곳에 설정해 놓은 순서대로 작업함(원하는 경우 순서 변경을 할 수 있음)
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    # agents에는 순서 상관 없이 사용될 agent를 넣어주면 됨
    agents=[
        international_chef,
        healthy_chef,
    ],
    # 콘솔에서 어떤 일이 벌어지고 있는지 확인할 수 있음
    verbose=True,
)

# CrewAI는 invoke 대신 kickoff를 사용
result = crew.kickoff(
    # 값은 Dictionary로 전달해주면 됨
    # inputs={
    #     "dish": "Greek dinner",
    #     "people": "5"
    # }
    # dict 함수를 사용하여 쉽게 넣어 줄 수 있음
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

# Agent: International Chef
## Task: Come up with a Greek dinner that serves 5 people.


# Agent: International Chef
## Final Answer: 
### Ingredients Required (Serves 5)

**For the Greek Moussaka:**
- 2 medium eggplants, sliced into ½ inch rounds
- 1 pound ground lamb or beef
- 1 large onion, finely chopped
- 2 cloves garlic, minced
- 1 can (14 oz) crushed tomatoes
- 2 tablespoons tomato paste
- 1 teaspoon ground cinnamon
- 1 teaspoon dried oregano
- Salt and pepper to taste
- 4 medium potatoes, peeled and sliced into ¼ inch rounds
- 3 cups béchamel sauce (see below for ingredients)
- ½ cup grated parmesan cheese

**For the Béchamel Sauce:**
- 4 tablespoons unsalted butter
- 4 tablespoons all-purpose flour
- 2 cups whole milk
- 1 egg yolk
- ¼ teaspoon nutmeg
- Salt and pepper to taste

### Preparation Instructions

1. **Prepare the Eggplants:**
   - Sprinkle the sliced eggplants with salt and let them sit for about 30 minutes to draw out excess moisture. Rinse and pat them dry with a 